**Komentari**

Ostavljat ću komentare dok punimo bilježnicu da znamo u svakom trenutku što je što. Kasnije možemo lagano ih samo obrisati za završnu verziju.

Zasad ubacujem podatke u bilježnicu i malo ću poraditi s njima te ih rasporediti lijepo u tablice da možemo lagano raditi s njima što želimo.

**Podaci**
--

Dodajemo sve biblioteke koje će nam trebati (možda treba pozvati %pip install za slučaj da na računalu nisu instalirane)

In [1]:
import pandas as pd

Skidanje podataka s našeg repozitorija na GitHub-u i njihovo lokalno spremanje

In [7]:
def skini(url: str, datoteka: str):
    import urllib.request as rq, pathlib
    with rq.urlopen(url) as konekcija:
        pathlib.Path(datoteka).write_bytes(konekcija.read())
        
test_podaci = 'https://raw.githubusercontent.com/jbrigljevic/str_uc/main/Podaci/test.csv'
train_podaci = 'https://raw.githubusercontent.com/jbrigljevic/str_uc/main/Podaci/train.csv'
skini(test_podaci, 'test_podaci.csv')
skini(train_podaci, 'train_podaci.csv')

Učitavanje u Dataframe

In [17]:
df_test = pd.read_csv('test_podaci.csv')
df_train = pd.read_csv('train_podaci.csv')

In [18]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


Slijede skupovi ključnih riječi i lokacija te lista stringova koji sadrže same textove tweetova.

In [16]:
popis_keyword = {keyword for keyword in df_test['keyword']}
popis_keyword

{'ablaze',
 'accident',
 'aftershock',
 'airplane%20accident',
 'ambulance',
 'annihilated',
 'annihilation',
 'apocalypse',
 'armageddon',
 'army',
 'arson',
 'arsonist',
 'attack',
 'attacked',
 'avalanche',
 'battle',
 'bioterror',
 'bioterrorism',
 'blaze',
 'blazing',
 'bleeding',
 'blew%20up',
 'blight',
 'blizzard',
 'blood',
 'bloody',
 'blown%20up',
 'body%20bag',
 'body%20bagging',
 'body%20bags',
 'bomb',
 'bombed',
 'bombing',
 'bridge%20collapse',
 'buildings%20burning',
 'buildings%20on%20fire',
 'burned',
 'burning',
 'burning%20buildings',
 'bush%20fires',
 'casualties',
 'casualty',
 'catastrophe',
 'catastrophic',
 'chemical%20emergency',
 'cliff%20fall',
 'collapse',
 'collapsed',
 'collide',
 'collided',
 'collision',
 'crash',
 'crashed',
 'crush',
 'crushed',
 'curfew',
 'cyclone',
 'damage',
 'danger',
 'dead',
 'death',
 'deaths',
 'debris',
 'deluge',
 'deluged',
 'demolish',
 'demolished',
 'demolition',
 'derail',
 'derailed',
 'derailment',
 'desolate',
 'de

In [19]:
popis_location = {location for location in df_test['location']}
popis_location

{nan,
 'The Open road!',
 'Nap town',
 'Hell',
 'Guayaquil, Ecuador',
 'Michigan',
 'LND Greatest City In the World',
 'ÌÏT: 35.353272,-78.553744',
 'Franklinton - BR - Houston',
 'Baker, Louisiana',
 'NoPa, San Francisco',
 'Mackem in Bolton',
 'Northeast, Ohio',
 'The Land of Pleasant Living',
 'Brazil',
 '8.27.14 & 7.28.15',
 'Twitter.',
 'OUTERSPACE',
 'Salem, OR',
 'Bikini Bottom',
 'Mainer missing Guatemala',
 'Honduras',
 'K-Town ',
 'CA, AZ & NV',
 'Indianapolis, Indiana',
 'Cramerton, NC',
 'Italy',
 'TAIZ - YEMEN',
 'University of Oklahoma',
 'somewhere in time',
 'Shanatic/GGian/SRKian',
 'Beal Feirste Northern Ireland',
 'real world',
 'West Coast, Canada',
 'rest easy angel @datboiiharri.',
 'Somewhere in the Internet...',
 'everywhere',
 'Camberville (Bostonish)',
 'Atlanta Georgia ',
 'Playing: HL2: EP1, Dust: AET',
 'alberta, canada',
 'No',
 'trinity nc',
 'Montana/North Carolina',
 'Brussels, Belgium',
 'Greece',
 '@KEYTNC3 @KCOY',
 'Colorado, The Mile High City',
 "Z